In [ ]:
from collections import defaultdict
import re

In [ ]:
filename = "day16.example.input"
filename = "day16.example2.input"
filename = "day16.input"

with open(filename) as file:
    file_parts = file.read().split("\n\n")

In [ ]:
rules = defaultdict(list)
for name, *intervals in re.findall("([\w ]+): (\d+-\d+) or (\d+-\d+)", file_parts[0]):
    for interval in intervals:
        low, high = interval.split("-")
        rules[name].append((int(low), int(high)))

rules

In [ ]:
tickets = []
for ticket in re.findall("([\d,]+)", file_parts[2]):
    tickets.append(tuple(map(int, ticket.split(","))))

# Part 1

In [ ]:
def number_fulfills_rule(number, intervals):
    for (low, high) in intervals:
        if low <= number <= high:
            return True
    # We got here because none of the intervals matched
    return False

In [ ]:
def number_fulfills_at_least_one_rules(number, rules):
    for name, intervals in rules.items():
        if number_fulfills_rule(number, intervals):
            return True
    # We got here because none of the rules were fulfilled
    return False

In [ ]:
invalid_values = []
for ticket in tickets:
    for number in ticket:
        if not number_fulfills_at_least_one_rules(number, rules):
            invalid_values.append(number)

sum(invalid_values)

# Part 2

## Sort out which rule matches which field

In [ ]:
valid_tickets = []
for ticket in tickets:
    for number in ticket:
        if not number_fulfills_at_least_one_rules(number, rules):
            break
    else:
        # All numbers on this ticket fulfills at least one rule
        valid_tickets.append(ticket)

In [ ]:
# For each rule, there might be multiple fields where all the tickets fulfills the rule

possible_fields = defaultdict(set)
for name, intervals in rules.items():
    for field in range(len(rules)):
        if all(number_fulfills_rule(ticket[field], intervals) for ticket in valid_tickets):
            # This field on all tickets is valid for the rule "name"
            possible_fields[name].add(field)

In [ ]:
# Figure out which rule field belongs to which rule,
# by starting with a rule with only one possible field.

assigned_fields = dict()
while possible_fields:

    # Get a rule-name with only one possible field
    name = min(possible_fields, key=lambda x: len(possible_fields[x]))
    if len(possible_fields[name]) == 1:
        field = possible_fields[name].pop()
    else:
        raise ValueError("Impossible to solve")
        
    # Assign this field to the rule-name, and mark it as resolved
    assigned_fields[name] = field
    del possible_fields[name]
    
    # Update remaining possibilites by removing the newly assigned field
    for name in possible_fields:
        possible_fields[name].discard(field)

assigned_fields

## Find solution to puzzle

In [ ]:
# Get my ticket
for ticket in re.findall("([\d,]+)", file_parts[1]):
    my_ticket = tuple(map(int, ticket.split(",")))

In [ ]:
import math

math.prod(my_ticket[assigned_fields[name]] for name in rules if name.startswith("departure"))